In [30]:
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification

In [65]:
checkpoint_path = '/home/ksaputa/mspace/plotkarzyna/models/checkpoint-2080'
model = AutoModelForTokenClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, return_tensors='pt')

In [66]:
id2label = {
    0: "O",
    1: "B",
    2: "I",
    3: "E",
    4: "S"
}

In [67]:
tokenizer("Ala ma kota, który wszedł na drzewo.")

{'input_ids': [0, 37, 2121, 2185, 24112, 1947, 2377, 12745, 1998, 16621, 1899, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [69]:
text = "Ala ma kota, który wszedł na drzewo."
text = "Pogoda pod koniec tygodnia nie będzie miała dla nas litości. W dalszym ciągu spodziewamy się upałów. Miejscami temperatura zdecydowanie przekroczy 30 st. C i tylko lokalnie orzeźwienie przyniosą burze."
text = "Koalicja Obywatelska już odkryła karty i wiemy, jakie nazwiska znajdą się na listach w poszczególnych okręgach podczas jesiennych wyborów do Sejmu. Teraz czas na Lewicę. Partia ujawnia swoich kandydatów."

tokenized = tokenizer([text], return_tensors='pt')
pred = model(
    **tokenized
).logits.argmax(-1)[0]

' '.join([
    f"{tokenizer.decode(tok)} ({id2label[int(el)]})" for el, tok in zip(pred, list(tokenized['input_ids'][0]))
])


'<s> (E) Koalicja (B) Obywatelska (E) już (O) odkry (O) ła (O) karty (S) i (O) wiemy (S) , (O) jakie (B) nazwiska (E) znajdą (O) się (O) na (O) listach (B) w (I) poszczególnych (B) okręgach (E) podczas (O) jesie (B) nnych (B) wyborów (I) do (I) Sejmu (S) . (O) Teraz (O) czas (O) na (O) Lewi (S) cę (S) . (O) Partia (S) ujawnia (O) swoich (B) kandydatów (E) . (O) </s> (E)'

In [64]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el